<a href="https://colab.research.google.com/github/NayanNair18/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Install and import libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub pandas-gbq

In [6]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
import kagglehub
import pandas_gbq
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [24]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "big-data-analysis-472319"
assert PROJECT_ID != "YOUR_PROJECT_ID_HERE", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: big-data-analysis-472319


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [25]:
DATASET = "superstore_data"
TABLE   = "sales"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,Category,City,Country,Customer_ID,Customer_Name,Discount,Market,_________,Order_Date,Order_ID,...,Sales,Segment,Ship_Date,Ship_Mode,Shipping_Cost,State,Sub_Category,Year,Market2,weeknum
0,Office Supplies,Ajman,United Arab Emirates,PO-88653,Patrick O'Donnell,0.7,EMEA,1,2011-10-03 00:00:00.000,AE-2011-9160,...,83,Consumer,2011-10-07 00:00:00.000,Standard Class,5.69,'Ajman,Storage,2011,EMEA,41
1,Technology,Ajman,United Arab Emirates,PO-88653,Patrick O'Donnell,0.7,EMEA,1,2011-10-03 00:00:00.000,AE-2011-9160,...,78,Consumer,2011-10-07 00:00:00.000,Standard Class,3.87,'Ajman,Machines,2011,EMEA,41
2,Office Supplies,Wadi as Sir,Jordan,SW-102452,Scot Wooten,0.0,EMEA,1,2011-12-26 00:00:00.000,JO-2011-1740,...,53,Consumer,2012-01-02 00:00:00.000,Standard Class,7.52,'Amman,Art,2011,EMEA,53
3,Office Supplies,Wadi as Sir,Jordan,SW-102452,Scot Wooten,0.0,EMEA,1,2011-12-26 00:00:00.000,JO-2011-1740,...,31,Consumer,2012-01-02 00:00:00.000,Standard Class,2.96,'Amman,Storage,2011,EMEA,53
4,Furniture,Wadi as Sir,Jordan,SW-102452,Scot Wooten,0.0,EMEA,1,2011-12-26 00:00:00.000,JO-2011-1740,...,387,Consumer,2012-01-02 00:00:00.000,Standard Class,49.21,'Amman,Bookcases,2011,EMEA,53


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [26]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


Using Colab cache for faster access to the 'global-superstore' dataset.
Path to dataset files: /kaggle/input/global-superstore
Files:
- /kaggle/input/global-superstore/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [27]:
sql_str = """
SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  
Summarize what you learned here:

> **Notes:** The first line makes sure that it selects the sub category from the table. The keyword DISTINCT makes sure that there are no duplicates and only unique values. FROM specifiec the table from which we retrieve the data, referencing the sales table in the dataset from the GCP. The WHERE keyword filters it to only include those rows where the region is West.

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [28]:
sql_str = """
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Ship_Mode
ORDER BY order_count DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Standard Class,30775
1,Second Class,10309
2,First Class,7505
3,Same Day,2701


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [29]:
sql_str = """
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-209804,8981.3239
1,RB-193604,6976.0959
2,SC-200954,5757.4119
3,HL-150404,5622.4292
4,AB-101054,5444.8055


In [33]:
sql_str = """
SELECT Customer_Name, SUM(Profit) AS total_profit
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Customer_Name
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best_name = bq.query(sql_str).to_dataframe()
display(df_best_name)

,Customer_Name,total_profit
0,Tamara Chand,8672.89890
1,Raymond Buch,8453.04950
2,Sanjit Chand,8205.37990
3,Hunter Lopez,7816.56778
4,Bill Eplett,7410.00530


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

Prompt 1:

TASK: Generate a BigQuery SQL query.

CONTEXT: The table is '[YOUR_PROJECT_ID].superstore_data.sales`.

GOAL: Calculate the average discount for products in the Technology Category sold in the East region.

In [30]:
sql_str = """
SELECT AVG(Discount) AS avg_discount
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Category = 'Technology' AND Region = 'East'
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,avg_discount
0,0.143364


Prompt 2:

TASK: Generate a BigQuery SQL query.

CONTEXT: The table is '[YOUR_PROJECT_ID].superstore_data.sales`.

GOAL: Identify how many unique customers has each Segment (Consumer/Corporate/Home Office) served.

In [31]:
sql_str = """
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Segment
ORDER BY unique_customers DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,2509
1,Corporate,1457
2,Home Office,907


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  

First I was able to identify all the sub categories in the table as well as categorize the orders by shipping mode to see what is popular. I saw that Standard Class was the most used one followed by Second Class.

- **Investigate:** What alternate query or filter changed the story?

Assuming customers do not share the same first and last names, I was also able to group them by Customer_Name to properly identify the customers that provide the most profit.

- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?

I cross checked with the tables and sections in the Cloud and also ensured that the upload was successful through reinstating prompts and converting the file type.

- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?

In the business context, shareholder and marketing teams would be very interested in such data and information as it allows them to assess the different factors that drive their profit and how to reorganize their frameworks to improve sales.


## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.

# Task
Download the dataset "anandaramg/global-superstore" from Kaggle, load either "Global Superstore.csv" or "Orders.csv" into a BigQuery table named `sales`, and perform a sanity check by counting the rows in the table.

## Download dataset from kaggle

### Subtask:
Use `kagglehub.dataset_download` to download the specified dataset.


**Reasoning**:
Use the `kagglehub.dataset_download` function to download the dataset and store the path. Then print the path.



In [8]:
DATASET_REF = "anandaramg/global-superstore"
download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

Using Colab cache for faster access to the 'global-superstore' dataset.
Path to dataset files: /kaggle/input/global-superstore


## Identify the csv file

### Subtask:
Identify the correct CSV file (Global Superstore.csv or Orders.csv) within the downloaded dataset.


**Reasoning**:
List the files in the downloaded dataset directory to identify the correct CSV file.



In [9]:
p = pathlib.Path(download_path)
csv_file_path = None
for f in p.glob("**/*"):
    if f.is_file() and (f.name == "Global Superstore.csv" or f.name == "Orders.csv"):
        csv_file_path = str(f)
        print(f"Identified CSV file: {csv_file_path}")
        break

if csv_file_path is None:
    print("Could not find 'Global Superstore.csv' or 'Orders.csv' in the downloaded dataset.")

Could not find 'Global Superstore.csv' or 'Orders.csv' in the downloaded dataset.


**Reasoning**:
Since the expected CSV files were not found in the initial search, list all files in the downloaded path to understand the directory structure and available files.



In [10]:
p = pathlib.Path(download_path)
print("Files found in the downloaded dataset:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)

Files found in the downloaded dataset:
- /kaggle/input/global-superstore/Global Superstore.txt


## Identify the csv file

### Subtask:
Identify the correct CSV file (Global Superstore.csv or Orders.csv) within the downloaded dataset.


## Identify the csv file

### Subtask:
Identify the correct CSV file (Global Superstore.csv or Orders.csv) within the downloaded dataset.


**Reasoning**:
List the files in the downloaded dataset directory to verify their names and identify the correct CSV file.



# Task
Download the dataset "anandaramg/global-superstore" from Kaggle, convert the "Global Superstore.txt" file within the dataset to a CSV, load the CSV into a BigQuery table named "sales", and perform a sanity check by counting the rows in the table.

## Identify the text file

### Subtask:
Identify the correct text file (Global Superstore.txt) within the downloaded dataset.


**Reasoning**:
Iterate through the files in the downloaded dataset directory and identify the 'Global Superstore.txt' file.



In [11]:
p = pathlib.Path(download_path)
text_file_path = None
for f in p.glob("**/*"):
    if f.is_file() and f.name == "Global Superstore.txt":
        text_file_path = str(f)
        print(f"Identified text file: {text_file_path}")
        break

if text_file_path is None:
    print("Could not find 'Global Superstore.txt' in the downloaded dataset.")

Identified text file: /kaggle/input/global-superstore/Global Superstore.txt


## Convert text to csv

### Subtask:
Read the text file and convert it into a pandas DataFrame, then save it as a CSV file.


**Reasoning**:
Read the text file into a pandas DataFrame and save it as a CSV file.



In [12]:
try:
    df_superstore = pd.read_csv(text_file_path, delimiter='\t', encoding='latin-1')
    csv_file_path = "Global_Superstore.csv"
    df_superstore.to_csv(csv_file_path, index=False)
    print(f"Successfully converted {text_file_path} to {csv_file_path}")
except FileNotFoundError:
    print(f"Error: The file {text_file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Successfully converted /kaggle/input/global-superstore/Global Superstore.txt to Global_Superstore.csv


## Load csv into bigquery

### Subtask:
Load the CSV data from the converted "Global_Superstore.csv" file into a BigQuery table named `sales` within the specified project and dataset.


**Reasoning**:
Load the pandas DataFrame into a BigQuery table using pandas_gbq.to_gbq().



**Reasoning**:
The previous attempt to load the data failed due to an invalid column name in the DataFrame. The error message indicates that the column name "è®°å½æ°" is not allowed in BigQuery. To fix this, I need to clean the column names in the DataFrame before loading it to BigQuery.



In [14]:
import re

def clean_col_names(df):
    """Cleans column names of a pandas DataFrame for BigQuery compatibility."""
    cols = df.columns
    new_cols = []
    for col in cols:
        # Replace invalid characters with underscores
        new_col = re.sub(r'[^a-zA-Z0-9_]', '_', col)
        # Ensure the column name starts with a letter or underscore
        if not re.match(r'[a-zA-Z_]', new_col):
            new_col = '_' + new_col
        # Truncate to a reasonable length if necessary (BigQuery limit is 300)
        new_col = new_col[:300]
        new_cols.append(new_col)
    df.columns = new_cols
    return df

# Clean the column names
df_superstore = clean_col_names(df_superstore)

# Now try loading the DataFrame into BigQuery again
pandas_gbq.to_gbq(
    df_superstore,
    destination_table=f'{DATASET}.{TABLE}',
    project_id=PROJECT_ID,
    if_exists='replace',
    location='US'
)

print(f"Successfully loaded data into BigQuery table: {PROJECT_ID}.{DATASET}.{TABLE}")

100%|██████████| 1/1 [00:00<00:00, 8612.53it/s]

Successfully loaded data into BigQuery table: big-data-analysis-472319.superstore_data.sales


## Perform a sanity check

### Subtask:
Execute a `SELECT COUNT(*)` query against the new BigQuery table to verify the data was loaded.


**Reasoning**:
Construct and execute a SQL query to count rows in the BigQuery table and display the result.



In [15]:
query = f"SELECT COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET}.{TABLE}`"
try:
    df_count = bq.query(query).to_dataframe()
    print(f"Successfully counted rows in {PROJECT_ID}.{DATASET}.{TABLE}.")
    display(df_count)
except Exception as e:
    print(f"Error counting rows in BigQuery table: {e}")


Successfully counted rows in big-data-analysis-472319.superstore_data.sales.


,row_count
0,51290


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

In [16]:
df_superstore.head()

,Category,City,Country,Customer_ID,Customer_Name,Discount,Market,_________,Order_Date,Order_ID,...,Sales,Segment,Ship_Date,Ship_Mode,Shipping_Cost,State,Sub_Category,Year,Market2,weeknum
0,Office Supplies,Los Angeles,United States,LS-172304,Lycoris Saunders,0.0,US,1,2011-01-07 00:00:00.000,CA-2011-130813,...,19,Consumer,2011-01-09 00:00:00.000,Second Class,4.37,California,Paper,2011,North America,2
1,Office Supplies,Los Angeles,United States,MV-174854,Mark Van Huff,0.0,US,1,2011-01-21 00:00:00.000,CA-2011-148614,...,19,Consumer,2011-01-26 00:00:00.000,Standard Class,0.94,California,Paper,2011,North America,4
2,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,...,21,Consumer,2011-08-09 00:00:00.000,Standard Class,1.81,California,Paper,2011,North America,32
3,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,...,111,Consumer,2011-08-09 00:00:00.000,Standard Class,4.59,California,Paper,2011,North America,32
4,Office Supplies,Los Angeles,United States,AP-109154,Arthur Prichep,0.0,US,1,2011-09-29 00:00:00.000,CA-2011-146969,...,6,Consumer,2011-10-03 00:00:00.000,Standard Class,1.32,California,Paper,2011,North America,40


## Summary:

### Data Analysis Key Findings

*   The "Global Superstore.txt" file was successfully identified within the downloaded dataset.
*   The tab-delimited text file was successfully converted into a CSV file named "Global\_Superstore.csv".
*   The data from the CSV file was successfully loaded into the BigQuery table `big-data-analysis-472319.superstore_data.sales` after cleaning the column names to comply with BigQuery's requirements.
*   A sanity check by counting rows in the BigQuery table confirmed that 51290 rows were successfully loaded.

### Insights or Next Steps

*   Always validate column names for compatibility when loading data into BigQuery to avoid errors.
*   The BigQuery table is now ready for further analysis or querying.
